In [1]:
import numpy as np
import pandas as pd
import requests
import datetime
import matplotlib.pyplot as plt
from utils import utils
import torch

# Temperature data


In [3]:
api_key = 'WBmYeqGTZMphjzecIADAthdwDyQUcnbL'
# url = 'https://www.ncei.noaa.gov/cdo-web/api/v2/datasets'
url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND"
params = {
    "id": "GHCND",
    "name": "Daily Summaries",
    "datacoverage": 1,
    "startdate": "2018-01-01",
    "enddate": "2018-01-03"
}
headers = {
    'token': api_key
}
response = requests.get(url, headers=headers, params=params)
data = response.json()
df = pd.json_normalize(data, 'results')
print(df.head())

                  date datatype            station attributes  value
0  2018-01-01T00:00:00     TAVG  GHCND:AE000041196      H,,S,    186
1  2018-01-01T00:00:00     TMAX  GHCND:AE000041196       ,,S,    259
2  2018-01-01T00:00:00     TMIN  GHCND:AE000041196       ,,S,    112
3  2018-01-01T00:00:00     PRCP  GHCND:AEM00041194       ,,S,      0
4  2018-01-01T00:00:00     TAVG  GHCND:AEM00041194      H,,S,    209


In [ ]:
def get_noaa_data(api_key, station_id, start_date, end_date):
    base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    dataset = "GHCND"
    params = {
        "datasetid": dataset,
        "stationid": station_id,
        "startdate": start_date,
        "enddate": end_date,
        "units": "metric",
        "limit": 1000
    }
    headers = {
        "token": api_key
    }
    response = requests.get(base_url, headers=headers, params=params)
    return response.json()

In [ ]:
station_id = {
    "KARB" : ["GHCND:USC00200228"], # Ann Arbor Airport 
    "PANC" : "GHCND:USW00026451",
    "KBOI" : "GHCND:USW00024131",
    "KORD" : "GHCND:USW00094846",
    "KDEN" : "GHCND:USW00023062",
    "KDTW" : "GHCND:USW00014838",
    "PHNL" : "GHCND:USW00022521",


}

In [ ]:
new_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/stations"

# Jonathan's data and code

In [9]:
endpoint = "https://aviationweather.gov/api/data/metar"
stations = ["PANC"]  # and other stations too
params = {"ids": stations, "format": "json", "hours": 48}
response = requests.get(endpoint, params=params)
df = pd.DataFrame.from_records(response.json())

In [8]:
df["reportTime"] = pd.to_datetime(df["reportTime"])
df = df.loc[
    (df["reportTime"].dt.date == datetime.date(2023, 11, 19))  # or whatever the date is
    & (df["metarType"] == "METAR")
][["temp", "precip", "snow"]].agg(["mean", "max", "min", "count"])

y_temp = df["temp"][:3]
y_snow = df["snow"]["max"] > 0
y_rain = df["precip"]["max"] > 0

# Sahana time

In [17]:
endpoint = "https://aviationweather.gov/api/data/metar"
stations = ["PANC"]  # and other stations too
hours = 18766 # 1 year
params = {"ids": stations, "format": "json", "hours": hours}
response = requests.get(endpoint, params=params)
df = pd.DataFrame.from_records(response.json())

In [18]:
print(df["reportTime"].min())

2023-11-05 19:06:00


# Marcs meteostat nonsense

In [21]:
import meteostat
from meteostat import Hourly

In [25]:
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2018, 12, 31, 23, 59)

# Get hourly data
data = Hourly('KARBO', start, end)
data = data.fetch()


In [26]:
lat_lon = {
    "KARB" : (42.2231,-83.7453), # Ann Arbor Airport
    "PANC" : (61.1743,-149.9963), # Ted Stevens Anchorage International Airport
    "KBOI" : (43.5644,-116.2228), # Boise Air Terminal
    "KORD" : (41.9742,-87.9073), # Chicago O'Hare International Airport
    "KDEN" : (39.8561,-104.6737), # Denver International Airport
    "KDTW" : (42.2125,-83.3533), # Detroit Metropolitan Airport
    "PHNL" : (21.3187,-157.9225), # Honolulu International Airport
    "KIAH" : (29.9844,-95.3414), # George Bush Intercontinental Airport
    "KMIA" : (25.7933,-80.2906), # Miami International Airport
    "KMIC" : (45.0628,-93.3533), # Minneapolis Crystal Airport
    "KOKC" : (35.3931,-97.6008), # Will Rogers World Airport
    "KBNA" : (36.1244,-86.6782), # Nashville International Airport
    "KJFK" : (40.6397,-73.7789), # John F. Kennedy International Airport
    "KPHX" : (33.4342,-112.0117), # Phoenix Sky Harbor International Airport
    "KPWM" : (43.6461,-70.3092), # Portland International Jetport
    "KPDX" : (45.5886,-122.5975), # Portland International Airport
    "KSLC" : (40.7884,-111.9778), # Salt Lake City International Airport
    "KSAN" : (32.7336,-117.1897), # San Diego International Airport
    "KSFO" : (37.6189,-122.3750), # San Francisco International Airport
    "KSEA" : (47.4489,-122.3094), # Seattle Tacoma International Airport
    "KDCA" : (38.8522,-77.0378), # Ronald Reagan Washington National Airport
}

In [30]:
from meteostat import Stations, Hourly

stations = Stations()

start= datetime.datetime(2022, 11, 12)
end = datetime.datetime(2023, 11, 12)

for key, value in enumerate(lat_lon):
    stations = stations.nearby(lat_lon[value][0], lat_lon[value][1])
    print(stations.fetch(1))
    print(stations.fetch(1).index[0])
stations = stations.nearby(lat_lon["KARB"][0], lat_lon["KARB"][1])
hourly  = Hourly(stations.fetch(1).index[0], start, end)

hourly.fetch()

                         name country region   wmo  icao  latitude  longitude  \
id                                                                              
KARB0  Ann Arbor / Pittsfield      US     MI  <NA>  KARB    42.223   -83.7456   

       elevation         timezone hourly_start hourly_end daily_start  \
id                                                                      
KARB0      256.0  America/Detroit   2006-01-01 2023-11-17  2006-01-01   

       daily_end monthly_start monthly_end   distance  
id                                                     
KARB0 2022-04-24    2006-01-01  2022-01-01  27.090333  
KARB0
                                name country region   wmo  icao  latitude  \
id                                                                          
PALH0  Anchorage / Turnagain Heights      US     AK  <NA>  PALH     61.18   

       longitude  elevation        timezone hourly_start hourly_end  \
id                                                         

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
time,,,,,,,,,,,
2022-11-12 00:00:00,9.4,4.0,69.0,0.0,NaN,300.0,20.5,NaN,1005.8,NaN,3.0
2022-11-12 01:00:00,9.4,4.0,69.0,0.0,NaN,300.0,20.5,NaN,1005.8,NaN,3.0
2022-11-12 02:00:00,6.1,0.0,65.0,0.0,NaN,300.0,18.4,NaN,1007.2,NaN,3.0
2022-11-12 03:00:00,6.1,0.0,65.0,0.0,NaN,300.0,24.1,NaN,1007.2,NaN,3.0
2022-11-12 04:00:00,2.8,-2.1,70.0,0.0,NaN,290.0,22.3,NaN,1008.5,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...
2023-11-11 20:00:00,8.0,-1.5,51.0,0.0,NaN,350.0,9.0,NaN,1030.0,NaN,3.0
2023-11-11 21:00:00,7.2,-1.7,53.0,0.0,NaN,360.0,7.6,NaN,1031.0,NaN,3.0
2023-11-11 22:00:00,2.8,-1.7,72.0,0.0,NaN,360.0,5.4,NaN,1031.4,NaN,1.0


# Our data set

In [2]:
filename = "data/daily_data.csv"
df = pd.read_csv(filename, index_col=0)
df = df.reset_index()
print(df.head())

  station        date  temp_max  temp_mean  temp_min  rainfall  snow
0   72202  2022-11-12      28.9  25.841667      23.3      True  True
1   72202  2022-11-13      30.6  25.000000      21.1      True  True
2   72202  2022-11-14      30.0  25.750000      22.8     False  True
3   72202  2022-11-15      30.0  26.287500      23.3     False  True
4   72202  2022-11-16      30.6  25.933333      22.2     False  True


In [10]:
station_72202 = df[df['station'] == "72202"]
station_72202['date'] = pd.to_datetime(station_72202['date'])
test_date = datetime.datetime(2023, 9, 19)
train_data = station_72202[station_72202['date'] < test_date]
test_data = station_72202[station_72202['date'] >= test_date]
# print(station_72202.iloc[365])
station_72202_train = utils.WindowDataset(train_data["temp_min"], 7, 4)
station_72202_test = utils.WindowDataset(test_data["temp_min"], 7, 4)

/var/folders/pg/fsc_f00x29x9zx2dz6cl3sp00000gs/T/ipykernel_82570/138702599.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_72202['date'] = pd.to_datetime(station_72202['date'])


In [11]:
print(station_72202_train[0][1].shape)
train_loader = torch.utils.data.DataLoader(station_72202_train, batch_size=4, shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(station_72202_test, batch_size=4, shuffle=False, num_workers=2)

torch.Size([4])


In [5]:
X, y = next(iter(train_loader))
print(X.shape)

torch.Size([4, 7])


In [6]:
print(X.shape)
print(y.shape)

torch.Size([4, 7])
torch.Size([4, 4])


In [7]:
learning_rate = 5e-5
num_hidden_units = 16

model = utils.LSTM(input_size=7, hidden_size=num_hidden_units, output_size=4)
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
model(X).shape

torch.Size([4, 4])

In [9]:
utils.train_loop(model, train_loader,  optimizer, loss_function, epochs=100)

Epoch: 0, Loss: 544.8786690451882
Epoch: 10, Loss: 521.249989199948
Epoch: 20, Loss: 491.72142524223824
Epoch: 30, Loss: 470.82269415917335
Epoch: 40, Loss: 452.55845651998146
Epoch: 50, Loss: 435.1879725270457
Epoch: 60, Loss: 416.50144354089514
Epoch: 70, Loss: 399.77406400209895
Epoch: 80, Loss: 371.21620525013316
Epoch: 90, Loss: 348.4100255594625


In [12]:
model(test_loader)

AttributeError: 'DataLoader' object has no attribute 'dim'